In [1]:
import os

import numpy as np
import pandas as pd

from replay.metrics import NDCG, Recall, HitRate, Coverage, MAP
from replay.utils import convert2spark
from replay.session_handler import get_spark_session, State
from replay.experiment import Experiment


In [2]:
spark = get_spark_session()
state = State(spark)
spark.conf.set("spark.local.dir", "/home/baurzhan/tmp")
display(spark.sparkContext.getConf().getAll())



22/07/19 14:54:50 WARN Utils: Your hostname, recsys-automl resolves to a loopback address: 127.0.1.1; using 192.168.0.5 instead (on interface ens160)
22/07/19 14:54:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/19 14:54:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/19 14:54:51 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/07/19 14:54:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


[('spark.local.dir', '/home/baurzhan/tmp'),
 ('spark.sql.shuffle.partitions', '36'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', 'localhost'),
 ('spark.app.startTime', '1658242491175'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.driver.bindAddress', '127.0.0.1'),
 ('spark.driver.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true'),
 ('spark.driver.memory', '66g'),
 ('spark.driver.port', '39147'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.sql.warehouse.dir',
  'file:/home/baurzhan/model_comparison/spark-warehouse'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.kryoserializer.buffer.max', '256m'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.sql.execution.arrow.pyspark.enabled', 'true'),
 ('spark.driver.maxResultSize', '4g'),
 ('spark.app.id', 'local-1658242491760')]

In [3]:
DATA_NAME = "bookcrossing"
DATA_PATH = f"data/{DATA_NAME}/"

k = 10


In [4]:
test_ = pd.read_csv(f"{DATA_PATH}test.csv")
test = convert2spark(test_)

train_ = pd.read_csv(f"{DATA_PATH}train.csv")
train = convert2spark(train_)


In [5]:
e = Experiment(
    test=test,
    metrics={
        NDCG(): k,
        Recall(): k,
        Coverage(train): k,
        HitRate(): k,
        MAP(): k,
    },
)


22/07/19 14:54:55 WARN TaskSetManager: Stage 0 contains a task of very large size (1574 KiB). The maximum recommended task size is 1000 KiB.


In [6]:
for root, dirs, files in os.walk(f"{DATA_PATH}predicts", topdown=False):
    for name in files:
        path = os.path.join(root, name)
        print(path)
        file_name = path.split("/")[-1]
        data_ = convert2spark(pd.read_csv(path))
        e.add_result(file_name.split(".")[0], data_)


data/bookcrossing/predicts/implicit_als.csv


22/07/19 14:54:57 WARN TaskSetManager: Stage 3 contains a task of very large size (1574 KiB). The maximum recommended task size is 1000 KiB.


data/bookcrossing/predicts/replay_kNN_test_predict.csv


22/07/19 14:55:03 WARN TaskSetManager: Stage 33 contains a task of very large size (1574 KiB). The maximum recommended task size is 1000 KiB.


data/bookcrossing/predicts/replay_ALS_test_predict.csv


...

22/07/19 14:55:08 WARN TaskSetManager: Stage 90 contains a task of very large size (1574 KiB). The maximum recommended task size is 1000 KiB.


data/bookcrossing/predicts/pop_rec_test_predict.csv


22/07/19 14:55:10 WARN TaskSetManager: Stage 119 contains a task of very large size (1574 KiB). The maximum recommended task size is 1000 KiB.


data/bookcrossing/predicts/implicit_als_test_predict.csv


22/07/19 14:55:12 WARN TaskSetManager: Stage 149 contains a task of very large size (1574 KiB). The maximum recommended task size is 1000 KiB.
19-Jul-22 14:55:12, replay, WARNING: Recommendations contain items that were not present in the log. The resulting metric value can be more than 1.0 ¯\_(ツ)_/¯


In [7]:
#anime
e.results.sort_values("NDCG@10", ascending=False)


,Coverage@10,HitRate@10,MAP@10,NDCG@10,Recall@10
replay_SLIM_test_predict,0.287918,0.659091,0.055051,0.128453,0.113045
replay_kNN_test_predict,0.151788,0.626455,0.048636,0.115532,0.107773
implicit_als,0.111007,0.542182,0.042139,0.099243,0.088055
pop_rec_test_predict,0.004791,0.457273,0.028702,0.073075,0.068918
replay_MultVAE_test_predict,0.007595,0.453182,0.028246,0.072657,0.069200
implicit_als_test_predict,0.052816,0.426909,0.029013,0.072238,0.067045
replay_ALS_test_predict,0.100608,0.386727,0.027618,0.066382,0.063464
random_rec_test_predict,1.000000,0.011273,0.000322,0.001115,0.001136


In [7]:
# last fm
e.results.sort_values("NDCG@10", ascending=False)


,Coverage@10,HitRate@10,MAP@10,NDCG@10,Recall@10
replay_kNN_test_predict,0.021356,0.606182,0.041661,0.104433,0.094618
pop_rec_test_predict,0.000095,0.237818,0.010715,0.030939,0.028036
random_rec_test_predict,0.353608,0.000000,0.000000,0.000000,0.000000


In [7]:
# book crossing
e.results.sort_values("NDCG@10", ascending=False)


,Coverage@10,HitRate@10,MAP@10,NDCG@10,Recall@10
pop_rec_test_predict,0.000139,0.088337,0.013546,0.024383,0.014935
replay_kNN_test_predict,0.022708,0.104467,0.012669,0.024347,0.018244
implicit_als_test_predict,0.013774,0.086352,0.009695,0.019054,0.014554
replay_ALS_test_predict,0.016748,0.094789,0.008117,0.017739,0.015667
implicit_als,0.014894,0.084615,0.008474,0.017179,0.013732
random_rec_test_predict,0.170653,0.000496,0.000058,0.000116,0.000132
